<a href="https://colab.research.google.com/github/anwarfaraz/iamlazy/blob/main/Auto_query_parameterizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#paste your query here
example_sql ="""with geo_dim as ( select distinct territory_number, territory_number as geo_id, 'territory' as geo_code from z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl where territory_type_id = 'REP' and sales_force in ('AI2', 'AI3', 'AI2R', 'AI2A') ), geo_map as ( select distinct b.abbott_customer_id, b.sales_force_code, a.territory_number as geo_id, 'territory' as geo_code from z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a inner join ( select * from z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl where sales_force_code in ('AI2', 'AI3', 'AI2R', 'AI2A') and upper(franchise_name) = 'IMMUNOLOGY - RHEUM' ) b on a.territory_number = b.territory_number union all select distinct b.abbott_customer_id, b.sales_force_code, a.district as geo_id, 'district' as geo_code from z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a inner join ( select * from z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl where sales_force_code in ('AI2', 'AI3', 'AI2R', 'AI2A') and upper(franchise_name) = 'IMMUNOLOGY - RHEUM' ) b on a.territory_number = b.territory_number union all select distinct b.abbott_customer_id, b.sales_force_code, a.region as geo_id, 'region' as geo_code from z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a inner join ( select * from z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl where sales_force_code in ('AI2', 'AI3', 'AI2R', 'AI2A') and upper(franchise_name) = 'IMMUNOLOGY - RHEUM' ) b on a.territory_number = b.territory_number union all select distinct b.abbott_customer_id, b.sales_force_code, '-1' as geo_id, 'nation' as geo_code from z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a inner join ( select * from z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl where sales_force_code in ('AI2', 'AI3', 'AI2R', 'AI2A') and upper(franchise_name) = 'IMMUNOLOGY - RHEUM' ) b on a.territory_number = b.territory_number ), run_log as ( select franchise, max(id) as run_id from z_abv_cws_hcp360_brandanalytics_dm.run_log where upper(franchise) = 'IMMUNOLOGY - RHEUM' group by franchise ), base as ( select fact.sales_force_code, sales.sales_force_id as sales_force_id, geo_dim.geo_id, geo_dim.geo_code, fact.abbott_customer_id as physician_id, time.time_dimension_id as time_dimension_id, fact.franchise_name as franchise, sub_market_name as market_name, product_group_name as brand, channel_name, abbott_product_id as product_id, market_indication, metric_value, time_bucket_name from ( select * from z_abv_cws_hcp360_brandanalytics_dm.rltn_metric_fact_tbl WHERE UNIVERSE_NAME = '1VIEW' and upper(metric_name) IN ('NBRX_COUNT') ) fact inner join( select distinct territory_number, abbott_customer_id from z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl WHERE upper(UNIVERSE_NAME)= '1VIEW' AND upper(SALES_FORCE_CODE) IN ('AI2', 'AI3', 'AI2R', 'AI2A') AND UPPER(PDRP_FLAG)= 'N' ) phy on phy.abbott_customer_id = fact.abbott_customer_id and phy.territory_number = fact.geo_id inner join z_abv_cws_hcp360_brandanalytics_dm.sales_force_dim sales on fact.sales_force_code = sales.sales_force_name inner join ( select distinct child_product_id, sub_market_name, product_group_name, market_name from z_abv_cws_hcp360_brandanalytics_dm.dim_mdm_product_hier_wide_tbl where upper(market_name) in ( '1VIEW IMM RHEUM AS MARKET', '1VIEW IMM RHEUM PSA MARKET', '1VIEW IMM RHEUM RA MARKET' ) ) mkt on mkt.child_product_id = fact.abbott_product_id and mkt.market_name = fact.market_name inner join geo_dim on geo_dim.territory_number = fact.geo_id inner join ( select * from z_abv_cws_hcp360_brandanalytics_dm.time_dimension_tbl where kpi_group = 'NBRX_RHEUM' and time_bucket_name in ( 'C4W', 'P4W', 'C8W', 'P8W', 'C13W', 'P13W' ) ) time on fact.week_end_date >= time.start_date and fact.week_end_date <= time.end_date ), final as ( select distinct sales_force_code, base.sales_force_id, sum(metric_value) as metric_value, time_bucket_name, base.time_dimension_id as time_bucket_id, base.franchise as franchise, substring( market_name, locate(' ', market_name)+ 1 ) as market_name, base.physician_id from base where upper(BASE.MARKET_NAME) IN ( 'AS BIOLOGICS + NEW ORALS', 'PSA BIOLOGICS + NEW ORALS', 'RA BIOLOGICS + NEW ORALS' ) group by sales_force_code, base.sales_force_id, time_bucket_name, base.time_dimension_id, base.franchise, substring( market_name, locate(' ', market_name)+ 1 ), base.physician_id ) select calc.franchise, calc.geo_id, calc.geo_code, -1 as product_id, cast(null as string) as product_name, physician_id, 'NBRX_VOL_MKT_ABS_CNG' as metric_name, metric_value_abs as metric_value, market_name, time_bucket_id, data_refresh_date, run_id, -1 as market_id, cast(null as string) as segment_name, cast(null as string) as access_type, -1 as plan_id, cast('OVERALL' as string) as metric_type_group, cast(null as string) as metric_type_subgroup, sales_force_id, cast(null as string) as indication from ( select curr.sales_force_id as sales_force_id, curr.sales_force_code, curr.market_name, curr.time_bucket_name, cast(null as string) as geo_id, curr.franchise, curr.time_bucket_id as time_bucket_id, current_date() as data_refresh_date, cast(null as string) as geo_code, cast( ( curr.metric_value - prev.metric_value ) as decimal(32, 2) ) as metric_value_abs, case when prev.metric_value = 0 then 0.00 when ( curr.metric_value - prev.metric_value ) = 0 then 0.00 when prev.metric_value is null then 0.00 when ( curr.metric_value - prev.metric_value ) is null then 0.00 else cast( ( ( curr.metric_value - prev.metric_value ) \/ prev.metric_value ) * 100 as decimal(32, 2) ) end as metric_value_abs_pct, curr.physician_id from ( select * from final where time_bucket_name in ('C4W', 'C8W', 'C13W') ) curr inner join ( select * from final where time_bucket_name in ('P4W', 'P8W', 'P13W') ) prev on curr.sales_force_code = prev.sales_force_code and substring( curr.time_bucket_name, 2, length(curr.time_bucket_name) ) = substring( prev.time_bucket_name, 2, length(prev.time_bucket_name) ) and curr.market_name = prev.market_name and curr.sales_force_id = prev.sales_force_id and curr.sales_force_code = prev.sales_force_code and curr.franchise = prev.franchise and curr.physician_id = prev.physician_id ) calc inner join run_log on run_log.franchise = calc.franchise

"""

In [ ]:
#pip install sql_formatter

In [ ]:
from sql_formatter.core import format_sql

format=format_sql(example_sql)

In [ ]:
a=list(map(lambda x: x.strip(),format.splitlines()))


In [ ]:
#Enter your dictionary here
#make sure it has parenthesis included 
queryParams = queryParams ={ "{territory_number_param}" : "territory_type_id = 'REP'",
"{sales_force_param}" : "sales_force in ('AI2', 'AI3', 'AI2R', 'AI2A')",
"{sales_force_param_}":"sales_force_code in ('AI2', 'AI3', 'AI2R', 'AI2A')",
"{upper_sales_force_param_}":"upper(sales_force_code) in ('AI2', 'AI3', 'AI2R', 'AI2A')",
"{franchise_param}" : "upper(franchise) = 'IMMUNOLOGY - RHEUM'",
"{franchise_name_param}":"upper(franchise_name) = 'IMMUNOLOGY - RHEUM'",
"{UNIVERSE_NAME_param}" : "UNIVERSE_NAME = '1VIEW'",
"{metric_name_param}" : "upper(metric_name) IN ('NBRX_COUNT')",
"{market_name_param}" : "upper(market_name) in ('1VIEW IMM RHEUM AS MARKET', '1VIEW IMM RHEUM PSA MARKET', '1VIEW IMM RHEUM RA MARKET')",
"{kpi_group_param}" : "kpi_group = 'NBRX_RHEUM'",
"{BASE.MARKET_NAME_param}" : "upper(BASE.MARKET_NAME) IN ('AS BIOLOGICS + NEW ORALS', 'PSA BIOLOGICS + NEW ORALS', 'RA BIOLOGICS + NEW ORALS')",
"{upper_UNIVERSE_NAME_param}" : "upper(UNIVERSE_NAME) = '1VIEW'",
"{time_param}" : "time_bucket_name in ('C4W', 'P4W', 'C8W', 'P8W', 'C13W', 'P13W')",
"{time_param_1}" : "time_bucket_name in ('C4W', 'C8W', 'C13W')",
"{time_param_2}" : "time_bucket_name in ('P4W', 'P8W', 'P13W')",
"{segment_name_param}":"upper(segment_name) in ('RINVOQ_WRITER_FLAG', 'BIOLOGIC DECILE 12 MONTHS', 'SKYRIZI_WRITER_FLAG')",
"{segment_name_param_1}":"upper(segment_name) in ('JAK_IT_UP_PROFILE')",
"{segment_name_param_2}":"upper(segment_name) in ('PRIORITY_TARGET')",
"{pdrp_flag_param}":"upper(pdrp_flag) = 'N')"
} 

In [ ]:
for x,y in queryParams.items():
  print('"',x.replace("{","").replace("}",""),'"',":",'"',y,'"',",")

" territory_number_param " : " territory_type_id = 'REP' " ,
" sales_force_param " : " sales_force in ('AI2', 'AI3', 'AI2R', 'AI2A') " ,
" sales_force_param_ " : " sales_force_code in ('AI2', 'AI3', 'AI2R', 'AI2A') " ,
" upper_sales_force_param_ " : " upper(sales_force_code) in ('AI2', 'AI3', 'AI2R', 'AI2A') " ,
" franchise_param " : " upper(franchise) = 'IMMUNOLOGY - RHEUM' " ,
" franchise_name_param " : " upper(franchise_name) = 'IMMUNOLOGY - RHEUM' " ,
" UNIVERSE_NAME_param " : " UNIVERSE_NAME = '1VIEW' " ,
" metric_name_param " : " upper(metric_name) IN ('NBRX_COUNT') " ,
" market_name_param " : " upper(market_name) in ('1VIEW IMM RHEUM AS MARKET', '1VIEW IMM RHEUM PSA MARKET', '1VIEW IMM RHEUM RA MARKET') " ,
" kpi_group_param " : " kpi_group = 'NBRX_RHEUM' " ,
" BASE.MARKET_NAME_param " : " upper(BASE.MARKET_NAME) IN ('AS BIOLOGICS + NEW ORALS', 'PSA BIOLOGICS + NEW ORALS', 'RA BIOLOGICS + NEW ORALS') " ,
" upper_UNIVERSE_NAME_param " : " upper(UNIVERSE_NAME) = '1VIEW' " ,
" time

In [ ]:
import numpy as np
for x,y in queryParams.items():
  z=0
  for z in range(len(a)):
    if y.lower() not in a[z].lower():
      #print(x)
      a[z]=a[z]
    else:
      try:
          a[z]=a[z].lower().replace(y.lower(),x)
      except:
        print(x)
# check your query here
#print(np.array(a))

In [ ]:
final_query=" ".join(a)

In [ ]:
print(format_sql(final_query))

with geo_dim as (SELECT DISTINCT territory_number,
                                 territory_number as geo_id,
                                 'territory' as geo_code
                 FROM   z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl
                 WHERE  {territory_number_param}
                    and {sales_force_param}), geo_map as (SELECT DISTINCT b.abbott_customer_id,
                                                      b.sales_force_code,
                                                      a.territory_number as geo_id,
                                                      'territory' as geo_code
                                      FROM   z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a
                                          INNER JOIN (SELECT *
                                                      FROM   z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl
                                                      WHERE  {sales_force_p

In [ ]:
print(final_query)

with geo_dim as (SELECT DISTINCT territory_number, territory_number as geo_id, 'territory' as geo_code FROM   z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl where  {territory_number_param} and {sales_force_param}), geo_map as (select distinct b.abbott_customer_id, b.sales_force_code, a.territory_number as geo_id, 'territory' as geo_code FROM   z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a INNER JOIN (SELECT * FROM   z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl where  {sales_force_param_} and {franchise_name_param}) b ON a.territory_number = b.territory_number UNION all SELECT DISTINCT b.abbott_customer_id, b.sales_force_code, a.district as geo_id, 'district' as geo_code FROM   z_abv_cws_hcp360_brandanalytics_dm.rpt_rep_hierarchy_tbl a INNER JOIN (SELECT * FROM   z_abv_cws_hcp360_brandanalytics_dm.dim_physician_geo_universe_tbl where  {sales_force_param_} and {franchise_name_param}) b ON a.territory_number = b.territory_number UNION all SELECT

In [ ]:
param="'C4W', 'C13W', 'C8W','P4W','P8W','P13W', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'W17', 'W18', 'W19', 'W20', 'W21', 'W22', 'W23', 'W24', 'W25', 'W26'"
a=param.split(" ")
print("".join(a))

'C4W','C13W','C8W','P4W','P8W','P13W','W1','W2','W3','W4','W5','W6','W7','W8','W9','W10','W11','W12','W13','W14','W15','W16','W17','W18','W19','W20','W21','W22','W23','W24','W25','W26'
